In [ ]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import networkx as nx
from pathlib import Path
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', 500)

from sklearn.preprocessing import MinMaxScaler
import sys
sys.path.append('/Users/david/Dropbox/PhD/Scripts/Spatial analyses')
import pyspace
import libpysal as lps
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array
from esda import fdr
from importlib import reload
pd.set_option('display.max_rows', 500)
reload(pyspace)
import seaborn as sns
from esda.moran import Moran
# sns.set_theme(font = 'Helvetica')
%matplotlib inline
from numba import NumbaDeprecationWarning
import warnings
import os
# Suppress NumbaDeprecationWarning
warnings.filterwarnings("ignore", category=NumbaDeprecationWarning)
from utils import read_data, plot_covariate_distributions, plot_match, compare_balance, sizeof_fmt, optimize_memory_df, plot_categorical_proportional_diff, compute_mean_differences_and_proportions, love_plot, sensitivity_analysis_k_neighbors

In [ ]:
data_folder  = Path('../data/')
main_folder = Path('../manuscript/Economic analysis of integrative medicine/')
results_folder = main_folder/'output'
model_folder = results_folder/'models'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

## Treatment defitions

In [ ]:
len(['Methodes ayurvediques_n', "Methodes d'art therapie_n",
       'Methodes de massage_n', 'Methodes energetiques_n',
       'Methodes energetiques manuelles_n', 'Methodes hydrotherapeutiques_n',
       'Methodes occidentales_n', 'Methodes orientales_n',
       'Methodes prescriptives_n', 'Methodes psychologiques complementaires_n',
       'Methodes reflexes_n', 'Methodes therapeutiques par le mouvement_n',
       'Methodes ayurvediques_amount', "Methodes d'art therapie_amount",
       'Methodes de massage_amount', 'Methodes energetiques_amount',
       'Methodes energetiques manuelles_amount',
       'Methodes hydrotherapeutiques_amount', 'Methodes occidentales_amount',
       'Methodes orientales_amount', 'Methodes prescriptives_amount',
       'Methodes psychologiques complementaires_amount',
       'Methodes reflexes_amount',
       'Methodes therapeutiques par le mouvement_amount'])

In [ ]:
data = pd.read_parquet('../Data/processed/full_dataset_nonull.parquet.gzip', columns = ['uuid','gp','NOANNEE','NBAGE','age_group',
                                                                                        'CDPHYSSEXE','CDLANGUE','SEX_F','SEX_M','ssep3',
                                                                                        'ssep3_q','n_atc','LANG_DE','LANG_FR','LANG_EN',
                                                                                        'LANG_IT','D_MEDIC_B','D_MEDIC_S','mean_lst',
                                                                                        'mean_ndvi',
                                                                                        'mean_pm10',
                                                                                        'mean_pm25',
                                                                                        'mean_no2',
                                                                                        'mean_carnight',
                                                                                        'MTFRANCHISECOUV',
                                                                                        'MODEL_AH_STD',
                                                                                        'MODEL_HMO',
                                                                                        'MODEL_MF',
                                                                                        'MODEL_TEL',
                                                                                        'lon_masked',
                                                                                        'lat_masked',
                                                                                        'cds',
                                                                                        'PRESTATIONS_BRUTES_AOS',
                                                                                        'PRESTATIONS_BRUTES_LCA',
                                                                                        'PRESTATIONS_BRUTES_CAM',
                                                                                        'PRESTATIONS_TOTAL',
                                                                                        'PRESTATIONS_BRUTES_ATC',
                                                                                        'PRESTATIONS_BRUTES_AMBULATOIRE',
                                                                                        'PRESTATIONS_BRUTES_STATIONNAIRE',
                                                                                        'PRESTATIONS_ACCIDENT',
                                                                                        'PRESTATIONS_BIRTH',
                                                                                        'PRESTATIONS_DISEASE',
                                                                                        'treatment',
                                                                                        'treatment_lca_cam',
                                                                                        'n_month_lca_by_patient',
                                                                                        'n_dispensateur_lca',
                                                                                        'month_reached_1000_lca',
                                                                                        'CANTON_NAME','CANTON_ACRONYM',
                                                                                        'DEDUCTIBLE_0.0',
                                                                                        'DEDUCTIBLE_100.0',
                                                                                        'DEDUCTIBLE_200.0',
                                                                                        'DEDUCTIBLE_300.0',
                                                                                        'DEDUCTIBLE_400.0',
                                                                                        'DEDUCTIBLE_500.0',
                                                                                        'DEDUCTIBLE_600.0',
                                                                                        'DEDUCTIBLE_1000.0',
                                                                                        'DEDUCTIBLE_1500.0',
                                                                                        'DEDUCTIBLE_2000.0',
                                                                                        'DEDUCTIBLE_2500.0',
                                                                                        'Methodes ayurvediques_n', "Methodes d'art therapie_n",
                                                                                        'Methodes de massage_n', 'Methodes energetiques_n',
                                                                                        'Methodes energetiques manuelles_n', 'Methodes hydrotherapeutiques_n',
                                                                                        'Methodes occidentales_n', 'Methodes orientales_n',
                                                                                        'Methodes prescriptives_n', 'Methodes psychologiques complementaires_n',
                                                                                        'Methodes reflexes_n', 'Methodes therapeutiques par le mouvement_n',
                                                                                        'Methodes ayurvediques_amount', "Methodes d'art therapie_amount",
                                                                                        'Methodes de massage_amount', 'Methodes energetiques_amount',
                                                                                        'Methodes energetiques manuelles_amount',
                                                                                        'Methodes hydrotherapeutiques_amount', 'Methodes occidentales_amount',
                                                                                        'Methodes orientales_amount', 'Methodes prescriptives_amount',
                                                                                        'Methodes psychologiques complementaires_amount',
                                                                                        'Methodes reflexes_amount',
                                                                                        'Methodes therapeutiques par le mouvement_amount',
                                                                                        'n_inpatient_hosp',
                                                                                        'n_outpatient_hosp',
                                                                                        'n_month_inpatienthosp',
                                                                                        'n_month_outpatienthosp',
                                                                                        'time_to_rehosp_in',
                                                                                        'time_to_rehosp_out',
                                                                                        'month_reached_5000_aos',
                                                                                        'n_cat_dispensateur_aos',
                                                                                        'n_ss_cat_dispensateur_aos',
                                                                                        'Diabetes mellitus',
                                                                                        'Osteoporosis',
                                                                                        'Acid related disorders',
                                                                                        'Cardiovascular diseases',
                                                                                        'Hyperlipidemia',
                                                                                        'Cancer',
                                                                                        'Pain'])

data = gpd.GeoDataFrame(data, crs = 4326, geometry=gpd.points_from_xy(data.lon_masked, data.lat_masked))

data = data.to_crs(2056)
data['E'], data['N'] = data['geometry'].x, data['geometry'].y

In [ ]:
df_treated = optimize_memory_df(data[data.treatment.isnull()==False])
df_treated.columns = df_treated.columns.str.replace('\.0','',regex=True)

In [ ]:
df_amount_by_cdposition = pd.read_parquet('../Data/processed/Intermediate datasets/df_amount_by_cdposition.parquet.gzip')

In [ ]:
del data

In [ ]:
df_treated['year'] = df_treated['NOANNEE']-2016

In [ ]:
df_treated = pd.merge(df_treated,df_amount_by_cdposition, left_on = ['uuid','NOANNEE'], right_on = ['uuid','ANNEE_TRAITEMENT'], how='left')
df_treated[['Homeopathy','Neural therapy','Anthroposophic medicine','Phone consultation','Acupuncture','Phytotherapy','Traditional Chinese medicine']] = df_treated[['Homeopathy','Neural therapy','Anthroposophic medicine','Phone consultation','Acupuncture','Phytotherapy','Traditional Chinese medicine']].fillna(0)

In [ ]:
df_na_cols = df_treated.isna().sum()
df_na_cols[df_na_cols> 0]

## Create dataset by uuid

In [ ]:
df_uuid = df_treated[['uuid']]
df_uuid = df_uuid.drop_duplicates().reset_index(drop=True)

### Create cumulated variables

In [ ]:
ttmt_gp_uuid = df_treated.groupby('uuid')['treatment'].sum().to_dict()
df_treated['treatment_cumulated'] = df_treated['uuid'].map(ttmt_gp_uuid)

In [ ]:
ttmt_gp_uuid = df_treated.groupby('uuid')['treatment_lca_cam'].sum().to_dict()
df_treated['treatment_cumulated_lca_cam'] = df_treated['uuid'].map(ttmt_gp_uuid)

In [ ]:
ttmt_month_gp_uuid = df_treated.groupby('uuid')['n_month_lca_by_patient'].sum().to_dict()
df_treated['cumulated_n_month_lca_by_patient'] = df_treated['uuid'].map(ttmt_month_gp_uuid)

In [ ]:
df_uuid['cumulated_n_month_lca_by_patient'] = df_uuid['uuid'].map(ttmt_month_gp_uuid)

### Add "urbanicity" variable and language region

In [ ]:
import fiona
communes = gpd.read_file(data_folder/'raw/OFS/swissBOUNDARIES3D_1_5_LV95_LN02.gpkg', layer = 'tlm_hoheitsgebiet')


communes = communes[communes['icc'] == 'CH']

communes_typologie = pd.read_csv(data_folder/'raw/OFS/Typologie communes/SFSO_typologie_communes.csv', sep = ';')
communes_langues = pd.read_csv(data_folder/'raw/OFS/Typologie communes/SFSO_langues_communes.csv', sep = ';')

communes = pd.merge(communes, communes_langues[['GEO_NAME','DESC_VAL']], left_on = 'name', right_on = 'GEO_NAME', how = 'left').merge(communes_typologie[['GEO_NAME','DESC_VAL']], left_on = 'name', right_on = 'GEO_NAME', how = 'left')

communes = communes.drop(['GEO_NAME_x','GEO_NAME_y'], axis = 1).rename(columns={'DESC_VAL_x':"Language",'DESC_VAL_y':'Urbanicity'})

communes.loc[communes.Urbanicity.str.contains(' urbaine', regex=True, na=False),'Urbanicity_simple'] = 'Urban'
communes.loc[communes.Urbanicity.str.contains('périurbaine', regex=True, na=False),'Urbanicity_simple'] = 'Peri-urban'
communes.loc[communes.Urbanicity.str.contains('rural', regex=True, na=False),'Urbanicity_simple'] = 'Rural'

communes['pop_density'] = (communes['einwohnerzahl']/(communes.geometry.area/1000000))*10000

communes.loc[communes.name == 'Obersaxen Mundaun', 'Language'] = 'Région linguistique allemande'
communes.loc[communes.name == 'Herznach-Ueken', 'Language'] = 'Région linguistique allemande'
communes.loc[communes.name == 'Damphreux-Lugnez', 'Language'] = 'Région linguistique française'
communes.loc[communes.name == "Bois-d'Amont", 'Language'] = 'Région linguistique française'

lang_dict = {'Région linguistique allemande':'DE',
            'Région linguistique française':'FR',
            'Région linguistique italienne':'IT',
            'Région linguistique romanche':'RM'}

communes['Language'] = communes['Language'].map(lang_dict)

In [ ]:
lakes = gpd.read_file(data_folder/'raw/Lacs/g2s15.shp')
lakes = lakes.to_crs(2056)

In [ ]:
communes['language_cat'] = communes['Language'].map({'DE':"German",'FR':'French','IT':'Italian','RM':"Romansh"})

In [ ]:
communes[communes.name.isin(['Genève','Bâle','Zürich','Basel','Lausanne','Bern'])]['name']

In [ ]:
from matplotlib.patheffects import withStroke

fig, ax = plt.subplots(figsize = (12,12))
communes.plot('language_cat', categorical=True, legend=True,ax=ax, legend_kwds={"title":'Language region'})
gdf_names = communes[communes.name.isin(['Genève','Bâle','Zürich','Basel','Lausanne','Bern'])]
gdf_names.loc[gdf_names['name'] == 'Genève','name']='Geneva'
# gdf_names.apply(lambda x: ax.annotate(text=x['name'], xy=x.geometry.centroid.coords[0], ha='center', size=8), axis=1)
lakes.plot(color='lightgrey', ax=ax)
# Adding text annotations with white contour
for x, y, label in zip(gdf_names.geometry.centroid.x, gdf_names.geometry.centroid.y, gdf_names['name']):
    ax.text(x, y, label, fontsize=8, ha='right', va='bottom',
            path_effects=[withStroke(linewidth=3, foreground='white')])
ax.set_title('B', loc = 'left', size= 16)
ax.set_axis_off()  # Hide axes
plt.savefig(results_folder/'Figures_rapport/language_region.png', dpi = 300, bbox_inches='tight', transparent=True)

### Gérer les cantons manquants et les personnes ayant vécus dans plusieurs cantons

In [ ]:
cantons = gpd.read_file(data_folder/'raw/OFS/swissBOUNDARIES3D_1_5_LV95_LN02.gpkg', layer = 'tlm_kantonsgebiet')
cantons = cantons.to_crs(2056)

cantons['CANTON_NAME_manual'] = cantons['name']

# df_canton_na = df_treated_filtered[df_treated_filtered.CANTON_NAME.isnull()][['uuid','NOANNEE','CANTON_NAME','CANTON_ACRONYM','geometry']]
# Create a dictionary of Swiss cantons and their abbreviations
swiss_cantons = {
    "Aargau": "AG",
    "Appenzell Ausserrhoden": "AR",
    "Appenzell Innerrhoden": "AI",
    "Basel-Landschaft": "BL",
    "Basel-Stadt": "BS",
    "Bern": "BE",
    "Fribourg": "FR",
    "Genève": "GE",
    "Glarus": "GL",
    "Graubünden": "GR",
    "Jura": "JU",
    "Luzern": "LU",
    "Neuchâtel": "NE",
    "Nidwalden": "NW",
    "Obwalden": "OW",
    "St. Gallen": "SG",
    "Schaffhausen": "SH",
    "Schwyz": "SZ",
    "Solothurn": "SO",
    "Thurgau": "TG",
    "Ticino": "TI",
    "Uri": "UR",
    "Vaud": "VD",
    "Valais":"VS",
    "Zug": "ZG",
    "Zürich": "ZH"
}

cantons['CANTON_ACRONYM_manual'] = cantons['name'].map(swiss_cantons)

df_treated = gpd.sjoin_nearest(df_treated, communes[communes.objektart == 'Gemeindegebiet'][['name','pop_density','Urbanicity_simple','Language','geometry']]).drop('index_right', axis = 1).drop_duplicates()
df_treated = gpd.sjoin(df_treated, cantons[['CANTON_NAME_manual','CANTON_ACRONYM_manual','geometry']], predicate = 'within').drop('index_right', axis = 1).drop_duplicates()

baseline_cantons = df_treated.groupby('uuid')['CANTON_ACRONYM_manual'].first().to_dict()

df_treated['CANTON_ACRONYM_first'] = df_treated['uuid'].map(baseline_cantons)

df_treated = pd.concat([df_treated, pd.get_dummies(df_treated.Urbanicity_simple, prefix = 'urb', dtype=int),
               pd.get_dummies(df_treated.Language, prefix = 'region',dtype=int)], axis=1)

In [ ]:
from matplotlib.patheffects import withStroke

# Combine two colormaps
cmap1 = plt.get_cmap('tab20', 20)  # Get the first 20 colors
cmap2 = plt.get_cmap('tab20b', 20)  # Get another set of 20 colors
colors = np.vstack([cmap1(np.arange(20)), cmap2(np.arange(20))])
custom_cmap = plt.matplotlib.colors.ListedColormap(colors[:26, :])

fig, ax = plt.subplots(figsize = (12,12))


cantons.plot('name', categorical=True, legend=False,ax=ax, cmap=custom_cmap, legend_kwds={"title":'Language region'})
gdf_names = cantons.copy()
# gdf_names.loc[gdf_names['name'] == 'Genève','name']='Geneva'
# gdf_names.apply(lambda x: ax.annotate(text=x['name'], xy=x.geometry.centroid.coords[0], ha='center', size=8), axis=1)
lakes.plot(color='lightgrey', ax=ax)
# Adding text annotations with white contour
for x, y, label in zip(gdf_names.geometry.centroid.x, gdf_names.geometry.centroid.y, gdf_names['name']):
    ax.text(x, y, label, fontsize=8, ha='right', va='bottom',
            path_effects=[withStroke(linewidth=3, foreground='white')])
ax.set_title('A', loc = 'left', size= 16)
ax.set_axis_off()  # Hide axes
plt.savefig(results_folder/'Figures_rapport/cantons.png', dpi = 300, bbox_inches='tight', transparent=True)

## PCG Flags

Ref très utile sur données GM = clustering des patients sur base des PCGs: https://medinform.jmir.org/2022/4/e34274/PDF

In [ ]:
df_flag_aos = pd.read_parquet('../Data/processed/df_flag_aos_preprocessed.parquet.gzip')
df_flag_aos['n_flags'] = df_flag_aos.filter(regex='LOPCG.').sum(axis = 1)
df_flag_aos = df_flag_aos.rename({'LOPCG':'any_pcg'}, axis = 1)
df_flag_aos.columns = df_flag_aos.columns.str.replace('LOPCG','', regex=True).str.lower()

In [ ]:
df_flag_aos['Cancer_PCG'] = df_flag_aos[['kre', 'krk', 'kho']].any(axis=1).astype(int)
df_flag_aos['Inflammatory_PCG'] = df_flag_aos[['mcr', 'pso', 'rhe','msk']].any(axis=1).astype(int)
df_flag_aos['Diabetes_PCG'] = df_flag_aos[['dm1','dm2']].any(axis=1).astype(int)
df_flag_aos['Hypertension_related_PCG'] = df_flag_aos[['dmh', 'pah', 'hch']].any(axis=1).astype(int)
df_flag_aos['Immune_PCG'] = df_flag_aos[['aik', 'tra']].any(axis=1).astype(int)
df_flag_aos['Pain_PCG'] = df_flag_aos[['smc', 'smn']].any(axis=1).astype(int)
df_flag_aos['Thyroid_PCG'] = df_flag_aos[['thy']].any(axis=1).astype(int)
df_flag_aos['Mental_PCG'] = df_flag_aos[['dep','bsr','abh','adh','alz','psy']].any(axis=1).astype(int)
df_flag_aos['Asthma_PCG'] = df_flag_aos[['ast', 'cop']].any(axis=1).astype(int)
df_flag_aos['Parkinson_PCG'] = df_flag_aos[['par']].any(axis=1).astype(int)
df_flag_aos['Epilepsy_PCG'] = df_flag_aos[['epi']].any(axis=1).astype(int)
df_flag_aos['Glaucoma_PCG'] = df_flag_aos[['gla']].any(axis=1).astype(int)
df_flag_aos['HIV_AIDS_PCG'] = df_flag_aos[['hiv']].any(axis=1).astype(int)
df_flag_aos['Heart_disease_PCG'] = df_flag_aos[['car']].any(axis=1).astype(int)
df_flag_aos['Other_PCG'] = df_flag_aos[['nie','was','zfp','zns']].any(axis=1).astype(int)
df_flag_aos['Diseases_PCG'] = df_flag_aos[['Asthma_PCG',
 'Cancer_PCG',
 'Diabetes_PCG',
 'Epilepsy_PCG',
 'Glaucoma_PCG',
 'HIV_AIDS_PCG',
 'Heart_disease_PCG',
 'Hypertension_related_PCG',
 'Immune_PCG',
 'Inflammatory_PCG',
 'Mental_PCG',
 'Other_PCG',
 'Pain_PCG',
 'Parkinson_PCG',
 'Thyroid_PCG']].any(axis=1).astype(int)

### Create multimorbidity variable

In [ ]:
df_flag_aos.loc[df_flag_aos['n_flags'] < 2, 'multimorbidity'] = 0
df_flag_aos.loc[df_flag_aos['n_flags']>= 2, 'multimorbidity'] = 1

df_flag_aos.loc[df_flag_aos['n_flags'] < 1, 'healthy'] = 1
df_flag_aos.loc[df_flag_aos['n_flags']>= 1, 'healthy'] = 0

### Join Flags to main dataset

In [ ]:
df_treated = pd.merge(df_treated, df_flag_aos[['uuid','noannee','locdrhosp','any_pcg','n_flags','multimorbidity','Asthma_PCG',
 'Cancer_PCG',
 'Diabetes_PCG',
 'Epilepsy_PCG',
 'Glaucoma_PCG',
 'HIV_AIDS_PCG',
 'Heart_disease_PCG',
 'Hypertension_related_PCG',
 'Immune_PCG',
 'Inflammatory_PCG',
 'Mental_PCG',
 'Other_PCG',
 'Pain_PCG',
 'Parkinson_PCG',
 'Thyroid_PCG']], left_on = ['uuid','NOANNEE'], right_on = ['uuid','noannee'])

## Fill NAs 

In [ ]:
df_treated['n_inpatient_hosp'] = df_treated['n_inpatient_hosp'].fillna(0)
df_treated['n_outpatient_hosp'] = df_treated['n_outpatient_hosp'].fillna(0)
df_treated['PRESTATIONS_ACCIDENT'] = df_treated['PRESTATIONS_ACCIDENT'].fillna(0)
df_treated['PRESTATIONS_DISEASE'] = df_treated['PRESTATIONS_DISEASE'].fillna(0)
df_treated['PRESTATIONS_BIRTH'] = df_treated['PRESTATIONS_BIRTH'].fillna(0)
df_treated['n_dispensateur_lca'] = df_treated['n_dispensateur_lca'].fillna(0)
df_treated['PRESTATIONS_BRUTES_AOS'] = df_treated['PRESTATIONS_BRUTES_AOS'].fillna(0)
df_treated['PRESTATIONS_BRUTES_LCA'] = df_treated['PRESTATIONS_BRUTES_LCA'].fillna(0)
df_treated['PRESTATIONS_BRUTES_CAM'] = df_treated['PRESTATIONS_BRUTES_CAM'].fillna(0)
df_treated['PRESTATIONS_TOTAL'] = df_treated['PRESTATIONS_TOTAL'].fillna(0)
df_treated['PRESTATIONS_BRUTES_AMBULATOIRE'] = df_treated['PRESTATIONS_BRUTES_AMBULATOIRE'].fillna(0)
df_treated['PRESTATIONS_BRUTES_STATIONNAIRE'] = df_treated['PRESTATIONS_BRUTES_STATIONNAIRE'].fillna(0)
df_treated['Methodes de massage_amount'] = df_treated['Methodes de massage_amount'].fillna(0)
df_treated['Methodes energetiques_amount'] = df_treated['Methodes energetiques_amount'].fillna(0)
df_treated['Methodes occidentales_amount'] = df_treated['Methodes occidentales_amount'].fillna(0)
df_treated['Methodes hydrotherapeutiques_amount'] = df_treated['Methodes hydrotherapeutiques_amount'].fillna(0)
df_treated['Methodes orientales_amount'] = df_treated['Methodes orientales_amount'].fillna(0)
df_treated['n_cat_dispensateur_aos'] = df_treated['n_cat_dispensateur_aos'].fillna(0)
df_treated['n_ss_cat_dispensateur_aos'] = df_treated['n_ss_cat_dispensateur_aos'].fillna(0)
df_treated['n_month_inpatienthosp'] = df_treated['n_month_inpatienthosp'].fillna(0)
df_treated['n_month_outpatienthosp'] = df_treated['n_month_outpatienthosp'].fillna(0)

## Inverse Hyperbolic Sine transformation of costs

In [ ]:
# Apply the Inverse Hyperbolic Sine Transformation to the cost variable
# IHS transformation: y = ln(y + (y^2 + 1)^(1/2))
# Can also be done with numpy np.arcsinh
df_treated['PRESTATIONS_CAM_LCA'] = df_treated['PRESTATIONS_BRUTES_LCA'] + df_treated['PRESTATIONS_BRUTES_CAM']
df_treated['ihs_cost_lca_cam'] = np.arcsinh(df_treated['PRESTATIONS_CAM_LCA'])
df_treated['ihs_cost_aos'] = np.log(df_treated['PRESTATIONS_BRUTES_AOS'] + np.sqrt(df_treated['PRESTATIONS_BRUTES_AOS']**2 + 1))
df_treated['ihs_cost_aos_ambu'] = np.log(df_treated['PRESTATIONS_BRUTES_AMBULATOIRE'] + np.sqrt(df_treated['PRESTATIONS_BRUTES_AMBULATOIRE']**2 + 1))
df_treated['ihs_cost_aos_stat'] = np.log(df_treated['PRESTATIONS_BRUTES_STATIONNAIRE'] + np.sqrt(df_treated['PRESTATIONS_BRUTES_STATIONNAIRE']**2 + 1))
df_treated['ihs_cost_disease'] = np.log(df_treated['PRESTATIONS_DISEASE'] + np.sqrt(df_treated['PRESTATIONS_DISEASE']**2 + 1))
df_treated['ihs_cost_lca'] = np.log(df_treated['PRESTATIONS_BRUTES_LCA'] + np.sqrt(df_treated['PRESTATIONS_BRUTES_LCA']**2 + 1))
df_treated['ihs_cost_cam'] = np.log(df_treated['PRESTATIONS_BRUTES_CAM'] + np.sqrt(df_treated['PRESTATIONS_BRUTES_CAM']**2 + 1))
df_treated['ihs_cost_pain'] = np.log(df_treated['Pain'] + np.sqrt(df_treated['Pain']**2 + 1))
df_treated['ihs_cds'] = np.log(df_treated['cds'] + np.sqrt(df_treated['cds']**2 + 1))

## Standardize and dichotomize variables

In [ ]:
df_treated['non_zero_ihs_cost_aos'] = (df_treated['ihs_cost_aos'] > 0).astype(int)

In [ ]:
df_treated = pd.concat([df_treated, pd.get_dummies(df_treated.age_group, prefix = 'age', dtype=int)], axis=1)

df_treated.columns = df_treated.columns.str.replace('-','_')

df_treated['DEDUCTIBLE_abv_500'] = 1
df_treated.loc[df_treated.MTFRANCHISECOUV < 500, 'DEDUCTIBLE_abv_500'] = 0


# df_treated['D_MEDIC_S_cat'] = pd.cut(df_treated['D_MEDIC_S'], bins = [0, 5000, 15000, 50000])
# df_treated['D_MEDIC_B_cat'] = pd.cut(df_treated['D_MEDIC_B'], bins = [0, 5000, 15000, 50000])

### Log-transform variables

In [ ]:
df_treated['n_atc_log'] = np.log10(df_treated['n_atc']+1)
df_treated['n_inpatient_hosp_log'] = np.log10(df_treated['n_inpatient_hosp']+1)
df_treated['n_month_inpatienthosp_log'] = np.log10(df_treated['n_month_inpatienthosp']+1)
df_treated['D_MEDIC_B_log'] = np.log10(df_treated['D_MEDIC_B']+1)
df_treated['D_MEDIC_S_log'] = np.log10(df_treated['D_MEDIC_S']+1)
df_treated['cds_log'] = np.log10(df_treated['cds']+1)

In [ ]:
df_treated[df_treated.cds > 0].cds_log.plot.kde()

In [ ]:
df_treated[df_treated.PRESTATIONS_BRUTES_CAM> 0].filter(regex='uuid|NOANNEE|TOTAL|CAM|LCA|AOS')

In [ ]:
df_prestation_aos_cam = read_data(data_folder/'processed'/'df_prestation_aos_cam_preprocessed.parquet.gzip')

In [ ]:
df_prestation_aos_cam[df_prestation_aos_cam.uuid == '0003438d-8a67-42ec-93f3-ef928f9fd397'].groupby('ANNEE_TRAITEMENT').PRESTATIONS_BRUTES.sum()

In [ ]:
df_treated['PRESTATIONS_BRUTES_CAM_b'] = np.where(df_treated['PRESTATIONS_BRUTES_CAM'] > 0 ,1 ,0)
df_treated['PRESTATIONS_BRUTES_AOS_b'] = np.where(df_treated['PRESTATIONS_BRUTES_AOS'] > 0 ,1 ,0)
df_treated['PRESTATIONS_BRUTES_LCA_b'] = np.where(df_treated['PRESTATIONS_BRUTES_LCA'] > 0 ,1 ,0)
df_treated['PRESTATIONS_ACCIDENT_b'] = np.where(df_treated['PRESTATIONS_ACCIDENT'] > 0 ,1 ,0)
df_treated['PRESTATIONS_BIRTH_b'] = np.where(df_treated['PRESTATIONS_BIRTH'] > 0 ,1 ,0)
df_treated['PRESTATIONS_BRUTES_STATIONNAIRE_b'] = np.where(df_treated['PRESTATIONS_BRUTES_STATIONNAIRE'] > 0 ,1 ,0)

In [ ]:
df_treated.PRESTATIONS_BRUTES_CAM_b.value_counts()

In [ ]:
df_treated = df_treated.sort_values(['uuid','year'])

## Final variables

In [ ]:
df_treated['CAREMODEL'] = df_treated.filter(regex='MODEL_').idxmax(axis=1).str.replace('MODEL_', '')

In [ ]:
canton_to_language = {
    'Zürich': 'German',
    'Bern': 'German',
    'Luzern': 'German',
    'Uri': 'German',
    'Schwyz': 'German',
    'Obwalden': 'German',
    'Nidwalden': 'German',
    'Glarus': 'German',
    'Zug': 'German',
    'Fribourg': 'Mix',
    'Solothurn': 'German',
    'Basel-Stadt': 'German',
    'Basel-Landschaft': 'German',
    'Schaffhausen': 'German',
    'Appenzell Ausserrhoden': 'German',
    'Appenzell Innerrhoden': 'German',
    'St. Gallen': 'German',
    'Graubünden': 'German', # Note: Graubünden is trilingual with German, Romansh and Italian communities
    'Aargau': 'German',
    'Thurgau': 'German',
    'Ticino': 'Italian',
    'Vaud': 'French',
    'Valais': 'Mix', # Note: Valais is bilingual with French and German communities
    'Neuchâtel': 'French',
    'Genève': 'French',
    'Jura': 'French',
}

In [ ]:
df_treated['lang_region'] = df_treated['CANTON_NAME'].map(canton_to_language)

In [ ]:
df_treated[['region_DE','region_FR','region_IT','SEX_F', 'NBAGE','MODEL_AH_STD', 'MODEL_MF', 'MODEL_HMO', 'MODEL_TEL', 'ssep3_q', 'DEDUCTIBLE_300', 'DEDUCTIBLE_500', 'DEDUCTIBLE_1000', 'DEDUCTIBLE_1500', 'DEDUCTIBLE_2000', 'D_MEDIC_B_log', 'ihs_cost_aos', 'n_atc_log', 'n_month_inpatienthosp_log', 'locdrhosp', 'Asthma_PCG', 'Cancer_PCG', 'Diabetes_PCG', 'Epilepsy_PCG', 'Glaucoma_PCG', 'HIV_AIDS_PCG', 'Heart_disease_PCG', 'Hypertension_related_PCG', 'Immune_PCG', 'Inflammatory_PCG', 'Mental_PCG', 'Other_PCG', 'Pain_PCG', 'Parkinson_PCG', 'Thyroid_PCG', 'mean_no2', 'mean_ndvi', 'mean_carnight', 'urb_Peri_urban', 'urb_Urban']].corr().sort_values('region_FR')

### Update Swiss SEP levels

In [ ]:
df_treated.filter(regex='ss')

In [ ]:
df_treated['ssep3_q'] = df_treated['ssep3_q'].astype('float').astype('int').astype("string")
df_treated.loc[df_treated['ssep3_q'] == '1','ssep3_q'] = '1st - Lowest'
df_treated.loc[df_treated['ssep3_q'] == '2','ssep3_q'] = '2nd'
df_treated.loc[df_treated['ssep3_q'] == '3','ssep3_q'] = '3rd'
df_treated.loc[df_treated['ssep3_q'] == '4','ssep3_q'] = '4th'
df_treated.loc[df_treated['ssep3_q'] == '5','ssep3_q'] = '5th - Highest'

## Create dataset with individuals present for all 5 years

In [ ]:
# Get unique years
unique_years = set(df_treated['NOANNEE'])
# Group by patient_id and filter
df_treated_filtered = df_treated.groupby('uuid').filter(lambda x: set(x['NOANNEE']) == unique_years)

In [ ]:
df_treated_filtered['treatment_cam_only'] = 0
df_treated_filtered.loc[df_treated_filtered['PRESTATIONS_BRUTES_CAM']>0, 'treatment_cam_only'] = 1

In [ ]:
df_treated_filtered = df_treated_filtered.rename(columns = {'age_80+':'age_80plus'})


In [ ]:
df_treated_filtered.to_parquet(data_folder/'processed/df_treated_filtered.parquet.gzip', compression='gzip')

In [ ]:
df_treated.groupby('uuid').size().value_counts().plot.bar()

### Exclude minors

In [ ]:
id_minors = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.NBAGE < 19)].uuid
df_treated_filtered_nominors = df_treated_filtered[df_treated_filtered.uuid.isin(id_minors)==False]

### Create usage type categories

In [ ]:
# Initialize the 'usage_type' column to None
df_treated_filtered_nominors['usage_type'] = None

# Set 'usage_type' based on complex conditions
# Correcting the logical operator and ensuring each condition is in its own parentheses
df_treated_filtered_nominors.loc[(df_treated_filtered_nominors['PRESTATIONS_BRUTES_AOS'] > 0) & (df_treated_filtered_nominors['treatment_lca_cam'] == 1), 'usage_type'] = 'CAM (MHI or SI) & CM'

df_treated_filtered_nominors.loc[df_treated_filtered_nominors['treatment_lca_cam'] == 0, 'usage_type'] = 'CM only'

# Adjusting for the second complex condition
df_treated_filtered_nominors.loc[(df_treated_filtered_nominors['PRESTATIONS_BRUTES_AOS'] == 0) & (df_treated_filtered_nominors['treatment_lca_cam'] == 1), 'usage_type'] = 'CAM only (MHI or SI)'

In [ ]:
df_treated_filtered_nominors.to_parquet(data_folder/'processed/df_treated_filtered_nominors.parquet.gzip', compression='gzip')

## Create subgroup datasets

In [ ]:
df_treated_filtered = df_treated_filtered_nominors.copy()

In [ ]:
df_treated_filtered.groupby('multimorbidity')['ihs_cost_aos'].mean()

In [ ]:
# Healthy = People with PCG flags = 0 in any year
uuid_healthy = df_treated_filtered[df_treated_filtered.n_flags == 0].uuid.unique()
df_healthy = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_healthy)]

# Healthy = People with AOS = 0 at baseline
uuid_healthy_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.n_flags == 0)].uuid.unique()
df_healthy_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_healthy_bs)]

# Patients with multimorbidity
uuid_multimorbid = df_treated_filtered[df_treated_filtered.multimorbidity == 1].uuid.unique()
df_multimorbid = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_multimorbid)]

# Patients with multimorbidity at baseline
uuid_multimorbid_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.multimorbidity == 1)].uuid.unique()
df_multimorbid_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_multimorbid_bs)]

# Patients with cancer
uuid_cancer = df_treated_filtered[(df_treated_filtered.Cancer_PCG == 1)].uuid.unique()
df_cancer = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_cancer)]

# Patients with cancer at baseline
uuid_cancer_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.Cancer_PCG == 1)].uuid.unique()
df_cancer_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_cancer_bs)]

# Patients with diabetes
uuid_diab = df_treated_filtered[(df_treated_filtered.Diabetes_PCG == 1)].uuid.unique()
df_diab = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_diab)]

# Patients with diabetes at baseline
uuid_diab_bs = df_treated_filtered[(df_treated_filtered.NOANNEE==2017)&(df_treated_filtered.Diabetes_PCG == 1)].uuid.unique()
df_diab_bs = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_diab_bs)]

# Patients with pain
uuid_pain = df_treated_filtered[(df_treated_filtered.Pain_PCG == 1)].uuid.unique()
df_pain = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_pain)]

# Patients with pain
uuid_mental = df_treated_filtered[(df_treated_filtered.Mental_PCG == 1)].uuid.unique()
df_mental = df_treated_filtered[df_treated_filtered.uuid.isin(uuid_mental)]



In [ ]:
df_healthy.to_parquet(data_folder/'processed/df_healthy_nominors.parquet.gzip', compression='gzip' )
df_multimorbid.to_parquet(data_folder/'processed/df_multimorbidity_nominors.parquet.gzip', compression='gzip' )
df_cancer.to_parquet(data_folder/'processed/df_cancer_nominors.parquet.gzip', compression='gzip' )
df_diab.to_parquet(data_folder/'processed/df_diab_nominors.parquet.gzip', compression='gzip' )
df_pain.to_parquet(data_folder/'processed/df_pain_nominors.parquet.gzip', compression='gzip' )
df_mental.to_parquet(data_folder/'processed/df_mental_nominors.parquet.gzip', compression='gzip' )

In [ ]:
df_treated_filtered[df_treated_filtered.Cancer > 0 ].shape[0]

In [ ]:
df_multimorbid[df_multimorbid.NBAGE < 18]

In [ ]:
df_treated_filtered[df_treated_filtered.Cancer_PCG > 0].shape[0]

In [ ]:
df_treated_filtered[df_treated_filtered.Diabetes_PCG > 0].shape[0]

In [ ]:
df_treated_filtered[df_treated_filtered['Diabetes mellitus'] > 0].shape[0]

In [ ]:
df_treated_filtered[df_treated_filtered.Pain_PCG > 0].shape[0]

In [ ]:
for pcg in pcg_columns:
    n_w_pcg = df_treated_filtered[(df_treated_filtered[pcg] == 1)&(df_treated_filtered.NBAGE > 18)].uuid.nunique()
    print(pcg, n_w_pcg)

In [ ]:
df_treated_filtered.uuid.nunique()

In [ ]:
df_treated_filtered[(df_treated_filtered['n_flags'] > 0)].uuid.nunique()

In [ ]:
df_treated_filtered[(df_treated_filtered['n_flags'] == 0)].uuid.nunique()

In [ ]:
filtered_therap_clean = pd.read_csv(data_folder/'/processed/20230223_Therapies_ontology.csv',encoding='ISO-8859-1')

In [ ]:
'- '.join(filtered_therap_clean[filtered_therap_clean.Methode.isnull()==False].Methode.unique())

In [ ]:
[i for i in filtered_therap_clean.Methode.unique()]